<a href="https://colab.research.google.com/github/its0merfaruk/Comparing-the-performance-of-BERT-and-T5-models/blob/main/MTQuAD_Bert_FT_Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

# Veri setini yükleyin
dataset = load_dataset("incidelen/MedTurkQuAD")

# İlk birkaç veriyi inceleme
print(dataset['train'][0])  # Örnek bir veri


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.61k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/8.44M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{'context': "Apse (apse; abscess; abscessus), irinli yangı bölgesinde doku erimesi vardır, oluşan boşluğu irin doldurur. Genellikle piyojen bakterilerin neden olduğu fokal bir yangı türüdür. Vücudun herhangi bir yerinde bir doku ya da organda oluşan apse, kendini ağrı, kızarıklık ve şişikler yaparak belli eder. Bazı apseler kendiliğinden dışarı açılır ve akar. Apse'nin açılmadığı durumlarda bu işi ameliyatla yapmak gerekir. Absenin mutlak tedavisi drenaj, yani boşaltılmasıdır. Cerrahi girişimden önce antibiyotik uygulanması sepsisi önler.", 'question': 'Apse nedir ve nasıl oluşur?', 'answers': {'answer_start': [33], 'text': ['irinli yangı bölgesinde doku erimesi vardır, oluşan boşluğu irin doldurur']}}


In [13]:
from transformers import AutoTokenizer

# Tokenizer seçimi
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

# Örnek bir preprocess fonksiyonu
def preprocess_function(examples):
    # Tokenizer ile soru ve context'i işleyin
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True  # Offset bilgisi dahil
    )

    # start_positions ve end_positions hesaplama
    start_positions = []
    end_positions = []

    for idx in range(len(examples["context"])):
        start_char = examples["answers"][idx]["answer_start"][0]
        answer_text = examples["answers"][idx]["text"][0]
        end_char = start_char + len(answer_text)

        # Offset bilgileri ile token pozisyonlarını bulma
        offset_mapping = inputs["offset_mapping"][idx]
        token_start_index = None
        token_end_index = None

        for token_idx, (start, end) in enumerate(offset_mapping):
            if start <= start_char < end:
                token_start_index = token_idx
            if start < end_char <= end:
                token_end_index = token_idx

        # Pozisyon bulunamazsa padding değerine eşitle
        if token_start_index is None:
            token_start_index = tokenizer.model_max_length
        if token_end_index is None:
            token_end_index = tokenizer.model_max_length

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    del inputs["offset_mapping"]  # Offset bilgileri artık gerekmedi

    return inputs

# Tokenize etme
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6560 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForQuestionAnswering

# Modeli yükleme
model = AutoModelForQuestionAnswering.from_pretrained("dbmdz/bert-base-turkish-cased")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import DataCollatorWithPadding

# Veri doldurma
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# DataLoader'ı oluşturma
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-16-31a944ab965d>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.157100,1.098801
2,0.942700,1.063883
3,0.675400,1.104723


TrainOutput(global_step=2460, training_loss=1.1363190193486408, metrics={'train_runtime': 2099.3475, 'train_samples_per_second': 9.374, 'train_steps_per_second': 1.172, 'total_flos': 5142320172564480.0, 'train_loss': 1.1363190193486408, 'epoch': 3.0})

In [32]:
# Modeli ve tokenizer'ı yükleyin
model = AutoModelForQuestionAnswering.from_pretrained("./qa_model").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("./qa_model")

# Test (Demo)
#Aşağıdan bir soru seçip başındaki #'i kaldır.
#question = "Apse nedir?"
#question = "Apse nedir ve nasıl oluşur?"
#question = "Apse genellikle neyin neden olduğu bir yangı türüdür?"
#question = "Astım nedir?"
#question = "Astım atağının tedavisi nedir?"
#question = "Astım tedavisinde hangi cihazlar çoğunlukla kullanılır?"
context = '''
Apse (apse; abscess; abscessus), irinli yangı bölgesinde doku erimesi vardır, oluşan boşluğu irin doldurur. Genellikle piyojen bakterilerin neden olduğu fokal bir yangı türüdür. Vücudun herhangi bir yerinde bir doku ya da organda oluşan apse, kendini ağrı, kızarıklık ve şişikler yaparak belli eder. Bazı apseler kendiliğinden dışarı açılır ve akar. Apse'nin açılmadığı durumlarda bu işi ameliyatla yapmak gerekir. Absenin mutlak tedavisi drenaj, yani boşaltılmasıdır. Cerrahi girişimden önce antibiyotik uygulanması sepsisi önler.
Astım (bronşial astma), küçük bronşların ve bronşiollerin, çeşitli uyaranlara aşırı tepki vermesinin sonucu ortaya çıkan, solunum yolu daralmasına sebep olan kronik bir rahatsızlıktır. Astımın tipik karakteristiği bronşların mukoza ödemiyle daralmasından ötürü olan episodik dispnedir. Astımda belirtilerin aniden ortaya çıkmasına astım atağı veya astım krizi adı verilir. Astım atağının şiddet derecesine göre tedavisi değişir hafif astım atağında bronş genişleticiler nefes yolundan verilerek sonuç elde edilebilir. Astım tedavisinde çoğunlukla inhaler adı verilen medikal cihazlar kullanılır.
'''

inputs = tokenizer(question, context, return_tensors="pt").to("cuda")
outputs = model(**inputs)

# Cevabı çıkarma
start_index = outputs.start_logits.argmax()
end_index = outputs.end_logits.argmax()
answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index+1])
print(f"Cevap: {answer}")


Cevap: inhaler
